In [128]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import time as tm

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [129]:
%cd /content/drive/MyDrive
X_trn_1 = np.loadtxt( "train_challenges.dat" )
%cd /content/drive/MyDrive
Y_trn_1 = np.loadtxt( "train_responses.dat" )

/content/drive/MyDrive
/content/drive/MyDrive


In [130]:
mini_batch = 1400
Beta_array = np.zeros(7)
t_train = np.zeros(7)
error = np.zeros(7)

In [131]:
import random

In [132]:
regularization_constant = 0.001

In [133]:
p = np.linalg.lstsq(X_trn_1 , Y_trn_1 , rcond=None)[0]
beta = [0.70, 0.75, 0.80, 0.85, 0.90, 0.95, 1.00]

In [134]:
import numpy as np

In [135]:
# def hardthresholding(w):
#     w_copy = np.copy(w);
#     Kth_big = np.sort(w)[2048-512]
#     i=0;
#     while i<len(w):
#       if(w_copy[i]<Kth_big):
#         w_copy[i]=0;
#       i=i+1;
#     return w_copy;

In [136]:
def hardthresholding( w ):
    t = np.zeros_like( w )
    ind = np.argsort( abs( w ) )[ -512: ]
    t[ ind ] = w[ ind ]
    return t

In [137]:
def grad_1(Y_trn,X_trn,p):
  i=0
  sum_array = [0.0]*(len(X_trn[0]))
  while i<len(X_trn):
    a = Y_trn[i] - np.dot(p,X_trn[i])
    b = np.negative(X_trn[i])
    c = np.multiply(b,a)
    sum_array = np.add(sum_array , c)
    i=i+1
  mean=np.divide(sum_array,len(X_trn))
  return mean

def grad_2 (reg_constant, p) :               ##differential of the regularization summation term
   grad = [0.0]*(len(p))
   i = len (p) - 1
   while (i>=0):
     if p[i]>0:
       grad [i] = reg_constant
     else :
       grad [i] = -reg_constant
     i=i-1
   return grad

def grad_total(x,y):
   return np.add(x,y)

In [138]:
# def correctn(wt, X_trn,Y_trn ):                 ## Implementation of correction in Projected Gradient Descent
#   i=0
#   X_index = np.zeros(len(wt))
#   while i<len(wt):
#     if wt[i] !=0:
#       X_index[i] == 1
#     else:
#       X_index[i] == 0
#     i=i+1
#   X_trn_copy= X_trn





#   i=0
#   j=0
#   for i in range(len(Y_trn)):
#     for j in range(len(X_index)):
#         if X_index[j] == 0:
#             X_trn_copy[i][j] = 0

#   return np.linalg.lstsq( X_trn_copy, Y_trn, rcond = None )[0]

In [139]:
def correctn(p, X_train, Y_train):
    non_zero_indices = np.argsort(-np.abs(p))[:512]
    XI = X_train[:, non_zero_indices]
    u = np.linalg.lstsq(XI, Y_train, rcond=None)[0]
    updated_weights = np.zeros_like(p)
    updated_weights[non_zero_indices] = u
    p = updated_weights
    return p

In [140]:
def mymodel(X_trn,Y_trn,p,beta):
  v = np.zeros(len(p))      #Initialize v[0] for momentum method
  iteration = 0
  while(1):
#######
## Making the mini-batch of size 200 and storing it in X_trn and Y_trn

     numbers = list(range(0, 1600))
     num_samples = mini_batch

     random_numbers = random.sample(numbers, num_samples)

     X_trn = X_trn_1[random_numbers]
     Y_trn = Y_trn_1[random_numbers]

 #######

 #######
   ## Calculation of the gradient  at current iteration
     x = grad_1(Y_trn, X_trn, p)
     y = grad_2(regularization_constant, p)
     grad_t = grad_total(x,y)
########

#######
     w = np.add(np.multiply(beta,v) , np.multiply((1-beta),grad_t)) #momentum method to implement gradient-descent
#######

     t = np.subtract(p, w)     #update the p vector
     #w_t = hardthresholding(t)  #Sparsing it
     t_sparse = correctn(t,X_trn,Y_trn)   #Then applying correction function

     error_1 = np.linalg.norm(np.subtract(t_sparse,p))
     if (iteration == 22):                                #convergence criterion
        return t_sparse
     p = t_sparse          #updating the p vector
     v = w                 #updating the momentum vector
     iteration+=1

In [141]:
def mean_abs_error(Y_trn_1,X_trn_1,t):
  i=0
  error_2 = 0
  while i<len(Y_trn_1):
    y_exp = np.dot(t.T,X_trn_1[i]);
    error_2=error_2+np.abs(Y_trn_1[i]-y_exp)
    i=i+1
  return error_2/len(Y_trn_1)

In [142]:
def accuracy(Y_trn_1,X_trn_1,t):
  i=0
  error_2 = 0
  while i<len(Y_trn_1):
    y_exp = np.dot(t.T,X_trn_1[i]);
    error_2=error_2+(np.abs(Y_trn_1[i]-y_exp))/Y_trn_1[i]
    i=i+1
  return (100 - (error_2/len(Y_trn_1))*100)

In [143]:
%cd /content/drive/MyDrive/Colab Notebooks
X_test_1 = np.loadtxt( "dummy_test_challenges.dat" )
%cd /content/drive/MyDrive/Colab Notebooks
Y_test_1 = np.loadtxt( "dummy_test_responses.dat" )

/content/drive/MyDrive/Colab Notebooks
/content/drive/MyDrive/Colab Notebooks


In [144]:
i=0
while i<7:
  beta = Beta_array[i]
  tic = tm.perf_counter()
  t = mymodel(X_trn_1,Y_trn_1,p,beta)
  toc = tm.perf_counter()
  t_train[i] = toc - tic
  t = hardthresholding(t)
  error[i] = mean_abs_error(Y_trn_1,X_trn_1,t)
  accur_train = accuracy(Y_trn_1,X_trn_1,t)
  i=i+1

In [145]:
accur_test = accuracy(Y_test_1,X_test_1,t)

In [146]:
print(t_train)
print(error)

[6.71993373 5.45744529 4.22551621 6.80879523 5.79039427 4.17278781
 7.33617697]
[2.05405010e-07 2.03534186e-07 2.02415634e-07 2.05535055e-07
 2.02636532e-07 2.06073997e-07 2.04176497e-07]


In [147]:
import pandas as pd

# Assuming your data is stored in a DataFrame called 'df'
df.to_csv('data.csv', index=False)  # Save DataFrame to a CSV file


NameError: ignored